## Домашнее задание к лекции «Продвинутый pandas 2»

# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [76]:
ratings = pd.read_csv('ml-latest-small/ratings.csv', sep = ',')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [15]:
# группирую по пользователям с их количеством и переводим userId из индекса в столбец используя его для объединения
ratings_count = ratings.groupby('userId').count().reset_index()
ratings_count.head()

,userId,movieId,rating,timestamp
0,1,20,20,20
1,2,76,76,76
2,3,51,51,51
3,4,204,204,204
4,5,100,100,100


In [31]:
# фильтрую df по необходимым столбцам userId и rating
ratings_count = ratings_count[['userId','rating']]
ratings_count.head()

,userId,rating
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [21]:
# группирую только пользователей с ценкой более 100
rating_more_100 = ratings_count[ratings_count['rating']>100]
rating_more_100.head()

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [45]:
rating_max = ratings.groupby('userId').max().reset_index()
rating_min = ratings.groupby('userId').min().reset_index()

In [47]:
rating_max_timestamp = rating_max[['userId','timestamp']]
rating_min_timestamp = rating_min[['userId','timestamp']]

In [48]:
rating_max_timestamp.rename(columns = {'timestamp': 'timestamp_max'}, inplace = True)
rating_min_timestamp.rename(columns = {'timestamp': 'timestamp_min'}, inplace = True)

In [57]:
avg_timestamp = rating_more_100.merge(rating_max_timestamp, how = 'inner').merge(rating_min_timestamp, how = 'inner')
avg_timestamp.head()

,userId,rating,timestamp_max,timestamp_min
0,4,204,949982274,949778714
1,8,116,1154474527,1154389340
2,15,1700,1469330735,997937239
3,17,363,1127476640,1127468587
4,19,423,855195373,855190091


In [74]:
# функция рассчета

def average(row):
    result = (row['timestamp_max']-row['timestamp_min'])
    return result


In [75]:
avg_timestamp['avg'] = avg_timestamp.apply(average, axis = 1)
len(rating_more_100) == len(avg_timestamp)
avg_timestamp.head(10)

,userId,rating,timestamp_max,timestamp_min,avg
0,4,204,949982274,949778714,203560.0
1,8,116,1154474527,1154389340,85187.0
2,15,1700,1469330735,997937239,471393496.0
3,17,363,1127476640,1127468587,8053.0
4,19,423,855195373,855190091,5282.0
5,21,162,854522908,853157476,1365432.0
6,22,220,1131753381,1131661890,91491.0
7,23,726,1166728253,1148386124,18342129.0
8,26,172,1371811577,1351544316,20267261.0
9,30,1011,1060795346,944943070,115852276.0


# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:  
1. Таблицу с тремя типами выручки для каждого client_id без указания адреса клиента  
2. Аналогичную таблицу по типам выручки с указанием адреса клиента.  Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [4]:
import pandas as pd

In [5]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [6]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [7]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [8]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [97]:
# группирую данные rzd и auto

data = rzd.merge(auto, how = 'outer', on = 'client_id')
data.head(10)

,client_id,rzd_revenue,auto_revenue
0,111,1093.0,NaN
1,112,2810.0,NaN
2,113,10283.0,57483.0
3,114,5774.0,83.0
4,115,981.0,912.0
5,116,NaN,4834.0
6,117,NaN,98.0


In [99]:
# группирую данные rzd, auto и air
# Таблица с тремя типами выручки для каждого client_id без указания адреса клиента

data_full = data.merge(air, how = 'outer', on = 'client_id')
data_full.fillna(0).head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [101]:
# группирую данные data_full с client_base по client_id
# Таблица по типам выручки с указанием адреса клиента

data_all = data_full.merge(client_base, how = 'outer', on = 'client_id')
data_all.fillna(0).head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:


### 1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Набор координат это ширина и долгота в градусах,то есть можно использовать два числа до одного знака:coord = (12.9, 77.6),что бы определить локацию пользователя.

### 2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчета  пункта 1?

Возможно если расстояние исходя из координат не более например 50-70 км(пользователь в рамках города),то все его покупки суммировать в наиболее часто используемый набор координат.  
Если расстояние боллее рамок города,то координаты надо оставить и так же суммировать их для выяснения его интересов в рамках локации когда он уехал(командировка,отпуск)

 ### 3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Откуда и какие запросы поступают,исходя из этого можно определить дополнительные потребности пользователя и дать предложение.  Возможно определить как часто пользователь уезжает в связи с чем предложить допуслуги связанные с разъездной деятельностью.